# Step 6: One-Step Forecasting

In [ ]:
pred_62043 = output_62043.get_prediction(start=pd.to_datetime('2012-01-01'), dynamic=False)
pred_conf_62043 = pred_62043.conf_int()

In [ ]:
#plot observed values
ax = np.log(nyc_zip_62043['1996-04-01':]).plot(label='observed')

#plot predicted values
pred_62043.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.9)

#plot the range for confidence intervals
ax.fill_between(pred_conf_62043.index,
                pred_conf_62043.iloc[:, 0],
                pred_conf_62043.iloc[:, 1], color='g', alpha=.2)

#set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('House Values')
plt.legend()

plt.show()

# Step 7: Dynamic Forecasting

In [ ]:
#get the real and predicted values
top_nyc_62043_forecasted = pred_62043.predicted_mean
nyc_62043_truth = np.log(nyc_zip_62043['2012-01-01':]).values

#compute the mean square error
rmse_62043 = np.sqrt(((top_nyc_62043_forecasted - nyc_62043_truth) ** 2).mean())
print('The Root Mean Squared Error of our forecasts is {}'.format(round(rmse_62043, 2)))

In [ ]:
#get forecast 60 steps (5 years) ahead in future
prediction_62043 = output_62043.get_forecast(steps=61)

#get confidence intervals of forecasts
pred_conf_62043 = prediction_62043.conf_int()

In [ ]:
#plot future predictions with confidence intervals

ax = np.log(nyc_zip_62043).plot(label='observed', figsize=(20, 15))
prediction_62043.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_conf_62043.index,
                pred_conf_62043.iloc[:, 0],
                pred_conf_62043.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('House Values')

plt.legend()
plt.show()

In [ ]:
pred_dynamic_62043 = output_62043.get_prediction(start=pd.to_datetime('2017-01-01'), dynamic=True, full_results=True)
pred_dynamic_conf_62043 = pred_dynamic_62043.conf_int()

In [ ]:
for i , pred_dynamic in enumerate(pred_dynamics):
    series = timeseries[i][1]
    
    
    
    # Get the Real and predicted values
    series_forecasted = pred_dynamic[0].predicted_mean
    series_truth = series['2017-01-01':].squeeze()

    # series_truth.squeeze()
    # Compute the mean square error
    rmse = np.sqrt(((series_forecasted-series_truth)**2).mean())

    print(titles[i],'\n')
    print('The Root Mean Squared Error of our forecasts is {}'.format(round(rmse, 2)))
    print('The root mean squared error'.title()
          + 'of our forcase is'
          +'{}% of mean housing price from 2017.'.format(round(rmse/series_truth.mean()*100,3)))
    # The Mean Squared Error of our forecasts is 0.07
    
    
    ax = series['2015':].plot(label='observed', figsize=(20, 15))
    pred_dynamic[0].predicted_mean.plot(label='Dynamic Forecast', ax=ax)

    ax.fill_between(pred_dynamic[1].index,
                    pred_dynamic[1].iloc[:, 0],
                    pred_dynamic[1].iloc[:, 1], color='g', alpha=.3)

    ax.fill_betweenx(ax.get_ylim(), pd.to_datetime('2017-01-01'), series_forecasted.index[-1], alpha=.1, zorder=-1)

    ax.set_xlabel('Date')
    ax.set_ylabel('Housing Price')
    ax.set_title(titles[i])
    plt.legend()
    plt.show()

# Step 8: Interpreting Results (Actual v. Predicted)

In [ ]:
profits =[]
compare = []
for i, prediction in enumerate(forecasts):
   
    actual =  timeseries[i][1]['2018-04'].values[0][0] 
    future = prediction[0].predicted_mean['2021-04'].values[0]
    limits = prediction[1]['2020-04'].values[0]
    
    profit = future-actual
    compare.append(((i,titles[i]),round(profit,0),round(profit/actual*100,0), actual, future, limits))
    
    #sorting the profit margins by the relative quantities (% increase)
    compare_sorted_rel = sorted(compare, key = lambda x:x[2], reverse = True)

In [ ]:
topbar =[]
bottombar =[]
yerrs =[]
xs =[]
label =[]
for i in range(len(compare_sorted_rel)):
    topbar.append(round(compare_sorted_rel[i][4],0))
    bottombar.append(round(compare_sorted_rel[i][3],0))
    yerrs.append(round(compare_sorted_rel[i][-1][1]- compare_sorted_rel[i][-2],0))
    idx = compare_sorted_rel[i][0][0]
    label.append(str(compare_sorted_rel[i][2])[:-2]+'%')
    xs.append(str(timeseries[idx][0].RegionName.values[0]))

In [ ]:
plt.style.use('seaborn')
fig = plt.figure(figsize =(20,8))
x = range(1,len(topbar)+1)
plt.bar(x,topbar,yerr=yerrs, color = "red", label ='April 2021')
plt.bar(x, bottombar, color = "blue", label ='April 2018')

for i in range(len(label)):
    plt.text(x=x[i]+0.1,y=topbar[i]+500000, s=label[i],fontsize = 18)

plt.gcf().autofmt_xdate()
plt.legend(fontsize = 20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.ticklabel_format(style ='plain')
plt.title('Comparisons between actual price in April 2018 and predicted price in April 2021', fontsize=20)
plt.gcf().autofmt_xdate()

In [ ]:
for i in range(len(compare_sorted_rel)):
    print(compare_sorted_rel[i][0][1], '\t',label[i])